# F215089
## 20 January 2023
### Module Attendance Program

- The module attendance program imports the functions in Student_Att Notebook because the load_join_table and weekly_attendance function will be used in this program. 
- The last cell has the function(main_module_attendance) that makes use of all the functions in this program.
- It requires the senior tutor to input the  module code and week number to be monitored.

## Importing packages

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipynb
#import user-defined functions
from ipynb.fs.defs.Student_Att import *#import functions used in Student Att

##  Extracting module attendance information

In [2]:
def module_attendance(dfData, modulename, week_number):
    """
    This function calculates the percentage attendance recorded in the lecture venue, 
    the time of lecture and the type of lecture
    input:
        dfData: the dataframe
        modulename: the name of the module
        week_number: the week number
    return:
        dataframe showing the venue, time, type of lecture and percentage attendance
    """
    
    df_module_attendance = dfData.groupby(["week", "venue", "lecture_type", "time"]).agg("mean") * 100
    df_module_attendance ["attendance"] = df_module_attendance.sum(axis = 1)/ (len(df_module_attendance.columns) -1)
    df_module_attendance = df_module_attendance.round()
    df_module_attendance = df_module_attendance.iloc[:, -1].to_frame()
    df_module_attendance.reset_index(inplace = True)
    df_time =df_module_attendance["time"].str.split("-", n = 1, expand = True)
    df_module_attendance["Time"] = df_time[0]
    week = "W" + str(week_number)
    week_module_attendance = df_module_attendance.loc[df_module_attendance["week"] == week,\
                                                      ["Time","lecture_type", "venue", "attendance"]]
    week_module_attendance.rename(columns={"Time": "Time", "lecture_type": "Type", "venue": "Room",\
                        "attendance": "Attendance%"}, inplace = True) 
    return week_module_attendance

## Outputs

In [3]:
def colour_code(val):
    """setting the conditions for the colour code"""
    if val > 70:
        colour = "green"
    elif val > 50:
        colour = "yellow"
    else:
        colour = "red"
    styler=f'background-color: {colour}'
    return styler

def visualise_output(dfData):
    """
    applying formatting function to the dataframe.
    """
    styledOutput=dfData.style.hide_index().applymap(colour_code,\
                                                    subset = "Attendance%").format('{:.1f}', \
                                                                                    subset = "Attendance%")
    display(styledOutput)
    return styledOutput

## Applying functions on the given dataset

In [4]:
def main_module_att(modulename, week_number):
    #dealing with the COA111 dataframe
    if modulename == "COA111":
        dfCOA111 = load_join_table(sqlstr="""
        SELECT T.*, C.venue, C.week, C.lecture_type, C.time
        FROM COA111T T INNER JOIN COA111Sessions C
        ON T.sess_id = C.sess_id;
        """)
        dfCOA111Attendance = module_attendance(dfCOA111, "COA111", week_number)
        print(f"Attendance record for {modulename} Week {week_number}:")
        dfCOA111AttendanceVisual = visualise_output(dfCOA111Attendance)
    else:
        #dealing with the COA122 dataframe
        dfCOA122 = load_join_table(sqlstr="""
        SELECT T.*, C.venue, C.week, C.lecture_type, C.time
        FROM COA122T T INNER JOIN COA122Sessions C
        ON T.sess_id = C.sess_id;
        """)
        dfCOA122Attendance = module_attendance(dfCOA122, "COA122", week_number)
        print(f"Attendance record for {modulename} Week {week_number}:")
        dfCOA122AttendanceVisual = visualise_output(dfCOA122Attendance)

#calling main_module_att function
main_module_att("COA122", 1)

Attendance record for COA122 Week 1:


Time,Type,Room,Attendance%
14:00,Lecture,CC012,86.0
09:00,Computer Lab,N001...,42.0
11:00,Computer Lab,N001...,30.0
